In [1]:
#pragma cling load("./build/libkastore.so")
#pragma cling load("./build/libtskit.so")
#pragma cling add_include_path("./tskit/c")
#pragma cling add_include_path("./kastore/c")

#include "./kastore/c/kastore.h"
#include "./tskit/c/tskit.h"

#include <tskit.h>
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <vector>

#define check_tsk_error(val)                                                            \
    if (val < 0) {                                                                      \
        fprintf(stderr, "line %d: %s", __LINE__, tsk_strerror(val));                    \
        exit(EXIT_FAILURE);                                                             \
    }

In [2]:
#include <numeric>
#include <random>

#include <fstream>

#include <iostream>
#include <vector>
#include <utility>
#include <algorithm>
#include <iomanip>  // std::setprecision()

#include <new>

#include<map>

In [3]:
#define check_tsk_error(val)                                                            \
    if (val < 0) {                                                                      \
        fprintf(stderr, "line %d: %s", __LINE__, tsk_strerror(val));                    \
        exit(EXIT_FAILURE);                                                             \
    }


# Initialising mock data

In [4]:
// node collection

In [5]:
void sleepy_create_mock_nodes(tsk_node_table_t *self, int length=10) {
    for (int i=0; i<length; i++) {
        tsk_node_table_add_row(self, TSK_NODE_IS_SAMPLE, i, TSK_NULL, TSK_NULL, NULL, 0);
    }
}

In [6]:
// for testing
tsk_node_table_t nodes;
tsk_node_table_init(&nodes, 0);
sleepy_create_mock_nodes(&nodes);
tsk_node_table_print_state(&nodes, stdout)

# Initialising table data

In [7]:
void sleepy_init_tables(tsk_table_collection_t *self, int N=3, int m=1, int gen=1) {
    for (int i=0; i<m; i++) {
        for (int j=0; j<2*N; j++) {
            tsk_node_table_add_row(&self->nodes, TSK_NODE_IS_SAMPLE, (double) gen,
                                   TSK_NULL, TSK_NULL, NULL, 0);
        }
        gen++;
    }
}

In [8]:
int gen = 0;
tsk_table_collection_t tables;
tsk_table_collection_init(&tables, 0);

sleepy_init_tables(&tables, 3, 2, gen);
tables.nodes.num_rows

12

In [9]:
tsk_node_table_print_state(&tables.nodes, stdout)

# Reversing time function

In [10]:
void sleepy_reverse_time(tsk_node_table_t &nodes, int begin, int end = 0)
{
    std::vector<double> time_before; time_before.reserve(begin);
    std::vector<double> time_mid; time_before.reserve(begin+nodes.num_rows-end);
    std::vector<double> time_after;
    
    if (end == 0) end = nodes.num_rows;
    
    for (int i=0; i<begin; i++) time_before.emplace_back(nodes.time[i]);
    
    double max_element{0};
    for (int i=begin; i<end; i++) {
        if (nodes.time[i] >= max_element) {
            max_element = nodes.time[i];
        }
    }
    
    for (int i=begin; i<end; i++) {
        time_mid.emplace_back((nodes.time[i] - max_element) * (-1));
    }
    
    for (int i=end; i<nodes.num_rows; i++) {
        time_after.emplace_back(nodes.time[i]);
    }
    
    time_before.insert(std::end(time_before), std::begin(time_mid), std::end(time_mid));
    time_before.insert(std::end(time_before), std::begin(time_after), std::end(time_after));
    
    for (int i=0; i<nodes.num_rows; i++) {
        nodes.time[i] = time_before[i];
    }
    
}

In [11]:
// test completed

# sleepy_num_non_sample_node

In [12]:
tsk_id_t sleepy_num_non_sample_node(tsk_node_table_t &nodes) {
    tsk_id_t num_non_sample{0};
    for (int i=0; i<nodes.num_rows; i++) {
        if (nodes.flags[i] != 1) {
            num_non_sample++;
        }
    }
    return num_non_sample;
}

In [13]:
int gen = 0;
tsk_table_collection_t tables;
tsk_table_collection_init(&tables, 0);
sleepy_init_tables(&tables, 3, 2, gen);

In [14]:
tables.nodes.flags[0] = 0;
tables.nodes.flags[4] = 0;

In [15]:
sleepy_num_non_sample_node(tables.nodes)

2

In [16]:
tsk_node_table_print_state(&tables.nodes, stdout)

# Simplification process

In [17]:
struct MutationMetaData {
    tsk_id_t origin;
    double position;
};

In [18]:
void sleepy_simplify(tsk_table_collection_t &tables, std::vector<std::pair<tsk_id_t, MutationMetaData>> temp_mutations,
              tsk_id_t sample_generation)
{
    int ret;
    std::vector<tsk_id_t> samples;
    for (int i=0; i<tables.nodes.num_rows; i++) {
        if ((int) tables.nodes.time[i] < sample_generation) {
            samples.emplace_back(i);
        }
    }
    for (int i=0; i<temp_mutations.size(); i++) {
        tsk_site_table_add_row(&tables.sites, temp_mutations[i].second.position, "0", 1, NULL, 0);
        
        tsk_mutation_table_add_row(&tables.mutations, tables.sites.num_rows-1,
                                   temp_mutations[i].first, TSK_NULL, TSK_UNKNOWN_TIME , "1", 1, NULL, 0);
    }
    
    tsk_id_t* samples_array = &samples[0];
    ret = tsk_table_collection_sort(&tables, NULL, 0); 
    check_tsk_error(ret);
    ret = tsk_table_collection_simplify(&tables, samples_array, samples.size(), TSK_FILTER_SITES, NULL); 
    check_tsk_error(ret);

}

In [19]:
int gen = 0;
tsk_table_collection_t tables;
tsk_table_collection_init(&tables, 0);

sleepy_init_tables(&tables, 3, 3, gen);

In [20]:
tsk_node_table_print_state(&tables.nodes, stdout)

In [21]:
int m = 2;

In [22]:
tables.sequence_length = 1;

In [23]:
sleepy_simplify(tables, std::vector<std::pair<tsk_id_t, MutationMetaData>>{}, m);

In [24]:
tsk_node_table_print_state(&tables.nodes, stdout)

# initial iteration

In [25]:
tsk_id_t sleepy_num_non_sample_node(tsk_node_table_t &nodes) {
    tsk_id_t num_non_sample{0};
    for (int i=0; i<nodes.num_rows; i++) {
        if (nodes.flags[i] != 1) {
            num_non_sample++;
        }
    }
    return num_non_sample;
}

In [26]:
int gen = 0;
tsk_table_collection_t tables;
tsk_table_collection_init(&tables, 0);


tables.sequence_length = 1;
int N = 2;
int m = 2;
sleepy_init_tables(&tables, N, m, gen);
gen = m;

In [27]:
tsk_node_table_print_state(&tables.nodes, stdout)

In [28]:
sleepy_reverse_time(tables.nodes, 0);    
sleepy_simplify(tables, std::vector<std::pair<tsk_id_t, MutationMetaData>>{}, m);

In [29]:
tsk_node_table_print_state(&tables.nodes, stdout)

In [30]:
tsk_id_t next_offspring_index = tables.nodes.num_rows;
tsk_id_t num_non_sample = sleepy_num_non_sample_node(tables.nodes);
tsk_id_t first_parental_index = next_offspring_index - (2*N) - num_non_sample; // all all sample nodes

In [31]:
tsk_node_table_print_state(&tables.nodes, stdout)

In [32]:
std::random_device rd;

In [33]:
std::mt19937 generator(rd());

In [34]:
void stats_random_discrete(std::vector<int> &output, const std::vector<double> &weights, int n)
{
    std::discrete_distribution<> d(weights.begin(), weights.end());
    for (int i=0; i<n; i++) { output.emplace_back(d(generator)); }
}

In [35]:
#include <cassert>   

In [36]:
void sleepy_dormancy_weights(std::vector<double> &weights, double b, tsk_id_t m)
{
    /* assert don't work in jupyter-cpp kernel */
    assert(("germination rate needs to be in [1,0[ interval. ", b > 1 || b <= 0));
    
    for (int i=0; i<m; i++) {
        weights.emplace_back(b * pow((1-b), i-1));
    }
    double sum = std::accumulate(std::begin(weights), std::end(weights), 0.0);
    
    
    for (int i=0; i<weights.size(); i++) {
        weights[i] = weights[i] / sum;
        
        /* all weights will be zero if b == 1*/
        /* otherwise  weights[0] == -nan */
        
        if (b == 1.0) weights[i] = 0;
    }
}

In [37]:
int m = 5;
double b = 0.5;
std::vector<double> dorm_weights;
sleepy_dormancy_weights(dorm_weights, b, m);

In [38]:
dorm_weights

{ 0.51612903, 0.25806452, 0.12903226, 0.064516129, 0.032258065 }

In [39]:
void sleepy_dormancy_generation(std::vector<tsk_id_t> &dormancy_generations, std::vector<double> &weights, tsk_id_t N) 
{
    stats_random_discrete(dormancy_generations, weights, N);
}

In [40]:
void stats_random_real(double &output, int start, int end)
{
    std::uniform_real_distribution<> d(start, end);
    output = d(generator); 
}

In [41]:
struct recombination_event {
    double left;
    double right;
    tsk_id_t parent;
    tsk_id_t child;
};

In [42]:
void stats_random_poisson(int &output, double lambda)
{
    std::poisson_distribution<> d(lambda);
    output = d(generator);
}

In [43]:
template<typename T>
std::vector<T> MultiplyVec(std::vector<T> v, int n)
{
    std::vector<T> rv;
    rv.reserve(n * v.size());
    for (int i=0; i<n; i++){
        for (auto e : v){ rv.emplace_back(e); }
    }
    return rv;
} 

In [44]:
void sleepy_recombination_events(std::vector<recombination_event> &recombination_events, double r,
                       std::pair<tsk_id_t, tsk_id_t> parent_idxs, tsk_id_t next_offspring_id, double L)
{
    int nbreaks;
    stats_random_poisson(nbreaks, r);

    if (nbreaks == 0) {
                
        recombination_event re = {0, L, parent_idxs.first, next_offspring_id};
        recombination_events.emplace_back(re);
        
        
    } else {
        std::vector<double> b;
        b.reserve(nbreaks);
        int i{0};
        while (i < nbreaks){
            double p;
            stats_random_real(p, 0, L);
            b.emplace_back(p);
            i++;
        }
        std::sort(b.begin(), b.end());
        if (b.back() != L) { b.emplace_back(L); }
        if (*b.begin() != 0.0) { 
            b.insert(b.begin(), 0.0);
        } else {
            int tmp_parent_idx = parent_idxs.first;
            parent_idxs.first = parent_idxs.second;
            parent_idxs.second = tmp_parent_idx;
        }
        std::vector<int> p_idxs{parent_idxs.first, parent_idxs.second};
        std::vector<int> pgams = MultiplyVec(p_idxs, b.size() / 2);
        for (int i=0; i<b.size()-1; i++) {
            recombination_event re = {b[i], b[i+1], pgams[i], next_offspring_id};
            recombination_events.emplace_back(re);
            //std::cout <<  b[i] << " " << b[i+1] << " " <<  pgams[i] << std::endl;
        }  
    }
}

In [53]:
std::vector<recombination_event> recombination_events;
double r = 1;
std::pair<tsk_id_t, tsk_id_t> parent_idxs = {3, 4};
tsk_id_t next_offspring_id = 12;
double L = 10;

sleepy_recombination_events(recombination_events, r, parent_idxs, next_offspring_id, L)

In [54]:
void sleepy_print_recombination_event(recombination_event &re) 
{
    std::cout << "left " << re.left << " right " << re.right << " parent " << re.parent << " child " << re.child << std::endl;
}

In [55]:
for (auto re : recombination_events) sleepy_print_recombination_event(re)

left 0 right 2.00362 parent 3 child 12
left 2.00362 right 8.02828 parent 4 child 12
left 8.02828 right 10 parent 3 child 12


In [42]:
void sleepy_add_time(tsk_node_table_t &nodes, int by, int begin, int end = 0)
{
    std::vector<double> time_before; 
    time_before.reserve(begin);
    std::vector<double> time_mid; 
    time_before.reserve(begin+nodes.num_rows-end);
    std::vector<double> time_after;
    
    if (end == 0) end = nodes.num_rows;
    for (int i=0; i<begin; i++) time_before.emplace_back(nodes.time[i]);
    

    
    for (int i=begin; i<end; i++) {
        time_mid.emplace_back(nodes.time[i] + by);
    }
    
    for (int i=end; i<nodes.num_rows; i++) {
        time_after.emplace_back(nodes.time[i]);
    }
    
    time_before.insert(std::end(time_before), std::begin(time_mid), std::end(time_mid));
    time_before.insert(std::end(time_before), std::begin(time_after), std::end(time_after));
    
    for (int i=0; i<nodes.num_rows; i++) {
        nodes.time[i] = time_before[i];
    }
    
}

# interations

In [57]:
int num_generations = 3;
int gc = 3;
double b = 1;
std::vector<tsk_id_t> parents;
int ret = 0;
int L = 1;
int r = 1;
int N = 2;
int m = 2;
int gen = 0;

tsk_table_collection_t tables;
tsk_table_collection_init(&tables, 0);
tables.sequence_length = 1;


sleepy_init_tables(&tables, N, m, gen);

sleepy_reverse_time(tables.nodes, 0);    
sleepy_simplify(tables, std::vector<std::pair<tsk_id_t, MutationMetaData>>{}, m);

tsk_node_table_print_state(&tables.nodes, stdout);


gen = m;

for(int num_generation=0; num_generation<num_generations; num_generation++) {
    
    std::cout << "generation " << num_generation << std::endl;

    tsk_id_t next_offspring_index = tables.nodes.num_rows;
    tsk_id_t num_non_sample = sleepy_num_non_sample_node(tables.nodes);
    tsk_id_t first_parental_index = next_offspring_index - (2*N) - num_non_sample; /* num_non_sample will be 0 during first iteration */
    
    std::cout << "first_parental_index " << first_parental_index << std::endl;

    for (int c=0; c<gc; c++) {

        /* vector of weights representing the probability of choosing parent from generation */
        /* index 0 means previous generation */
        std::vector<double> dorm_weights; sleepy_dormancy_weights(dorm_weights, b, m);
        
        /* actual generation from where to choose the parent 0 meaning previous generation, up to m-1 */
        std::vector<tsk_id_t> dormancy_generations; sleepy_dormancy_generation(dormancy_generations, dorm_weights, 2*N);

        parents.clear();
        stats_random_discrete(parents, std::vector<double>(N, 1), 2*N);

        tsk_id_t i_genotype = 0;
        for (int p=0; p<parents.size(); p+=2) {

            /* way too complicated dormancy index updating dependent on state in the simplication process */
            tsk_id_t parent1 = parents[p];
            tsk_id_t parent2 = parents[p+1];
            tsk_id_t dormancy_updated_first_parental_index_1 = first_parental_index - (2*N*dormancy_generations[parent1]); 
            tsk_id_t dormancy_updated_first_parental_index_2 = first_parental_index - (2*N*dormancy_generations[parent2]); 
            if (c > 0 && (2*N*dormancy_generations[parent1]) >= c*2*N) { dormancy_updated_first_parental_index_1 -= num_non_sample; }
            if (c > 0 && (2*N*dormancy_generations[parent2]) >= c*2*N) { dormancy_updated_first_parental_index_2 -= num_non_sample; }
            tsk_id_t p1g1 = dormancy_updated_first_parental_index_1 + 2*parent1;
            tsk_id_t p1g2 = p1g1+1;
            tsk_id_t p2g1 = dormancy_updated_first_parental_index_2 + 2*parent2;
            tsk_id_t p2g2 = p2g1+1;
            

            double mendel1 = 0;
            double mendel2 = 0;
            stats_random_real(mendel1, 0, 1);
            stats_random_real(mendel2, 0, 1);
            if (mendel1 < 0.5) {
                tsk_id_t tmp = p1g1;
                p1g1 = p1g2;
                p1g2 = tmp;
            }
            if (mendel2 < 0.5) {
                tsk_id_t tmp = p2g1;
                p2g1 = p2g2;
                p2g2 = tmp;
            }

            tsk_node_table_add_row(&tables.nodes, TSK_NODE_IS_SAMPLE, gen, TSK_NULL, TSK_NULL, NULL, 0);
            tsk_node_table_add_row(&tables.nodes, TSK_NODE_IS_SAMPLE, gen, TSK_NULL, TSK_NULL, NULL, 0);
            
            tsk_node_table_print_state(&tables.nodes, stdout);


            std::vector<recombination_event> recombination_events;
            sleepy_recombination_events(recombination_events, r, std::pair<tsk_id_t, tsk_id_t>{p1g1,p1g2},next_offspring_index, L);

            for (int re=0; re<recombination_events.size(); re++) {
                ret = tsk_edge_table_add_row(&tables.edges, recombination_events[re].left, recombination_events[re].right, recombination_events[re].parent, recombination_events[re].child, NULL, 0);
            }

            i_genotype++;
            next_offspring_index++;

            recombination_events.clear();
            sleepy_recombination_events(recombination_events, r, std::pair<tsk_id_t, tsk_id_t>{p2g1,p2g2},next_offspring_index, L);

            for (int re=0; re<recombination_events.size(); re++) {
                ret = tsk_edge_table_add_row(&tables.edges, recombination_events[re].left, recombination_events[re].right, recombination_events[re].parent, recombination_events[re].child, NULL, 0);
            }

            i_genotype++;
            next_offspring_index++;
        }

        /* function is only relevant within simplification interval */
        /* first_parental_index will be updated differently after simplifcation*/
        first_parental_index = tables.nodes.num_rows - 2*N;

        gen++;
        //num_generation++; 
    }

    sleepy_add_time(tables.nodes, gc, 0, tables.nodes.num_rows-2*N*gc);
    sleepy_reverse_time(tables.nodes, tables.nodes.num_rows-2*N*gc, tables.nodes.num_rows);  
    sleepy_simplify(tables,std::vector<std::pair<tsk_id_t, MutationMetaData>>{}, m);
}

tsk_node_table_print_state(&tables.nodes, stdout);


generation 0
first_parental_index 4
generation 1
first_parental_index 4
generation 2
first_parental_index 4


# mendel switch

In [4]:
std::random_device rd;
std::mt19937 generator(rd());

In [5]:
void stats_random_real(double &output, int start, int end)
{
    std::uniform_real_distribution<> d(start, end);
    output = d(generator); 
}

In [6]:
void sleepy_random_gamete_switch(tsk_id_t &gamete1, tsk_id_t &gamete2){
    double mendel = 0;
    stats_random_real(mendel, 0, 1);
    if (mendel < 0.5) {
        tsk_id_t tmp = gamete1;
        gamete1 = gamete2;
        gamete2 = tmp;
    }
}

In [8]:
tsk_id_t g1 = 1, g2 = 3;

In [19]:
random_gamete_switch(g1, g2);

In [20]:
g1

1

In [21]:
g2

3